In [ ]:
import re
import os
import json
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import math
import time
import jieba
from rank_bm25 import BM25Okapi
from elasticsearch import Elasticsearch

# LLM
import torch
from transformers import pipeline

# line
from waitress import serve
from flask import Flask, request
from linebot import LineBotApi, WebhookHandler
from linebot.models import TextSendMessage
import threading

In [ ]:
class ELS():
    def __init__(self) -> None:
        self.basic_auth = ''
        self.position = ''
        self.query_keywords = None
        self.es = None
    
    # 建立 elastic 連結
    def login(self):
        self.es = Elasticsearch(
            [self.position],
            basic_auth=('elastic', self.basic_auth),
            verify_certs=False
        )

    # 載入關鍵字表
    def load_key_syn_list(self):
        # 讀取關鍵字文件
        with open('key_syn\keywords.txt', 'r', encoding='utf-8') as file:
            lines = file.readlines()
        self.query_keywords = [line.strip() for line in lines]

    # 時間分數遞減
    def calculate_time_weight(self, timestamp):
        # 抓離現在的時間遠近
        current_time = datetime.now()
        doc_time = datetime.strptime(timestamp, "%Y%m%d")
        days_diff = (current_time - doc_time).days
        
        if days_diff == 0:
            # 避免除以零
            days_diff = 1
        
        time_weight = 1 / (math.log(days_diff + 1, 2)+1)  # 避免 log(0) 的錯誤
        return time_weight
    
    # 中文切詞
    def tokenize(self, text):
        return list(jieba.cut(text))

    # 文本搜尋
    def search(self, index_name, scaned_keywords, content_keywords, category_keywords, txt, score_threshold=10):
        documents = []
        # 計算300天前的日期
        date_threshold = (datetime.now() - timedelta(days=300)).strftime('%Y-%m-%d')
        
        if scaned_keywords:
            # 針對重要關鍵詞進行查詢
            body = {
                "query": {
                    "bool": {
                        "must": [
                            {"range": {"timestamp": {"gte": date_threshold}}}
                        ],
                        "should": [{"match": {"keywords": keyword}} for keyword in self.query_keywords],
                        "minimum_should_match": 1
                    }
                }
            }
            
            # 關鍵字搜尋
            res = self.es.search(index=index_name, body=body)
            for hit in res['hits']['hits']:
                doc_content = hit["_source"]["content"]
                doc_timestamp = hit["_source"].get("timestamp", "1970-01-01")
                doc_keywords = hit["_source"].get("keywords", [])
                documents.append({
                    'content': doc_content,
                    'timestamp': doc_timestamp,
                    'keywords': doc_keywords,
                    'id': hit["_id"]
                })

        if not documents:
            return ' ', None, None
        
        # 從txt中提取出與self.query_keywords匹配的關鍵字
        tokenized_query = [keyword for keyword in self.query_keywords if keyword in txt]
        
        # 如果沒有匹配的關鍵字，返回'無相關內容'
        if not tokenized_query:
            return ' ', None, None

        # 對文本進行分詞
        tokenized_corpus = [self.tokenize(doc['content']) for doc in documents]
        
        # 建立 bm25
        bm25 = BM25Okapi(tokenized_corpus, k1=1, b=0.75)
        
        # 計算 bm25 分數
        all_scores = bm25.get_scores(tokenized_query)
        
        # 把分數和文本放一起
        scores = []
        for idx, doc in enumerate(documents):
            score = all_scores[idx]
            time_weight = self.calculate_time_weight(doc['timestamp'])
            final_score = score / time_weight
            scores.append((final_score, doc['content'], doc['timestamp'], doc['keywords'], doc['id']))

        # 初排（文本分數/時間遞減）
        scores.sort(reverse=True, key=lambda x: x[0])

        # 類別關鍵字優先排序
        category_priority_ranking = []
        non_category_ranking = []
        for score, content, timestamp, doc_keywords, doc_id in scores:
            if any(keyword in category_keywords for keyword in doc_keywords):
                # 匹配到的放 category_priority_ranking
                category_priority_ranking.append((score, content, timestamp, doc_keywords, doc_id))
            else:
                # 未匹配到的放 non_category_ranking
                non_category_ranking.append((score, content, timestamp, doc_keywords, doc_id))

        # 合併保持排序順序
        final_ranking = category_priority_ranking + non_category_ranking

        # 內容關鍵字次優先排序
        content_priority_ranking = []
        non_content_ranking = []
        for score, content, timestamp, doc_keywords, doc_id in final_ranking:
            if any(keyword in content_keywords for keyword in doc_keywords):
                # 匹配到的 content_priority_ranking
                content_priority_ranking.append((score, content, timestamp, doc_keywords, doc_id))
            else:
                # 未匹配到的 non_content_ranking
                non_content_ranking.append((score, content, timestamp, doc_keywords, doc_id))

        # 合併保持排序順序
        final_ranking = content_priority_ranking + non_content_ranking

        # 檢查分數是否低於閾值
        max_final_score = final_ranking[0][0] if final_ranking else 0
        best_document = final_ranking[0][1] if final_ranking else '無相關內容'
        time = final_ranking[0][2] if final_ranking else None

        # 加上時間
        best_document = f'資料時間:{time}\n' + best_document

        if max_final_score < score_threshold:
            return ' ', max_final_score, time
        
        return best_document, max_final_score, time
    
    def search_basic(self, index_name, matched_keywords):
        documents = []
        
        if matched_keywords:
            body = {
                "query": {
                    "bool": {
                        "should": [{"match": {"keywords": keyword}} for keyword in matched_keywords],
                        "minimum_should_match": 1
                    }
                },
                "size": 10
            }
            
            # 冠鍵字搜尋
            res = self.es.search(index=index_name, body=body)
            
            # 搜尋結果處理
            for hit in res['hits']['hits']:
                doc_content = hit["_source"]["content"]
                doc_timestamp = hit["_source"].get("timestamp", "2024-08-26")
                doc_score = hit["_score"]
                documents.append((doc_content, doc_timestamp, hit["_id"], doc_score))
        
        # 找不到文件 返回空字串
        if not documents:
            return ' '
        
        # 按 Elasticsearch 内建分数排序
        documents.sort(key=lambda x: x[3], reverse=True)
        
        # 找分數最高者
        best_document, best_timestamp, best_id, best_score = documents[0]
        
        return best_document

In [ ]:
# 客戶資料庫
class ClientDeport():
    def __init__(self) -> None:
        self.client_df = pd.read_csv('client_df.csv')
        self.ID_list = [os.path.splitext(filename)[0] for filename in os.listdir('client') if filename.endswith('.json')]

    # 取得客戶資料
    def get_client_df(self):
        self.client_df = pd.read_csv('client_df.csv')
    
    # 更新客戶資料總表
    def renew_client_df(self, name:str, number:str, user_ID:str):
        # 新增新資料
        new_data = {
            '姓名': name,
            '電話': number,
            '最後更新日期': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'user_ID': user_ID,
            '認證': '電腦認證'
        }
        # 新資料轉 df
        new_df = pd.DataFrame([new_data])
        # 新增到原本 df
        self.client_df = pd.concat([self.client_df, new_df], ignore_index=True)
        # 儲存 df
        self.client_df.to_csv('client_df.csv', index=False, encoding='utf-8')

    # ID 比對
    def ID_check(self, user_ID):
        if user_ID in self.client_df['user_ID'].values:
            auth_value = self.client_df.loc[self.client_df['user_ID'] == user_ID, '認證'].values[0]
            if auth_value == '電腦認證' or auth_value == '人工認證':
                return '比對成功'
            return '認證異常'
        else:
            return '尚無資料'
    
    # 姓名_電話 擷取
    def extract_name_phone(self, message:str):
        # 正則表達式
        pattern = r'(\S+?)\s*[\r\n\s]*([\d]{10})'
        
        # 看是否有符合項
        match = re.search(pattern, message)
        
        if match:
            # 擷取姓名 電話
            name = match.group(1)
            phone = match.group(2)
            return name, phone
        else:
            return None, None

    # 取得 ID_list
    def get_iso_data_list(self):
        self.ID_list = [os.path.splitext(filename)[0] for filename in os.listdir('client') if filename.endswith('.json')]

    # 創建客戶個人資料庫
    def bulid_iso_data(self, user_ID): # 每次有新建立，self裡面的iso_list就要更新，重刷一次
        # 初始化json
        data = {
            "last_update": "",
            "updates": []
        }
        
        # 創建新 json
        with open(f'client\{user_ID}.json', 'w') as json_file:
            json.dump(data, json_file, indent=4)

    # 更新客戶個人資料庫
    def renew_iso_data(self, user_ID, input_text, client_keywords):
        # 載入 json 文件
        if os.path.exists(f'client\{user_ID}.json'):
            with open(f'client\{user_ID}.json', 'r') as json_file:
                data = json.load(json_file)
        else:
            print(f'No {user_ID} json deport')
        
        # 得到當前時間
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        # 更新 last_update 時間
        data["last_update"] = current_time
        
        # 新的紀錄
        new_record = {
            "text": input_text,
            "time": current_time,
            "keywords": client_keywords
        }
        # 把新紀錄新增進去
        data["updates"].append(new_record)
        
        # 儲存更新後的 json
        with open(f'client\{user_ID}.json', 'w') as json_file:
            json.dump(data, json_file, indent=4)

In [ ]:
from transformers import pipeline, StoppingCriteria
class EosListStoppingCriteria(StoppingCriteria):
    def __init__(self, eos_sequence=[128256]):
        self.eos_sequence = eos_sequence

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        last_ids = input_ids[:, -len(self.eos_sequence):].tolist()
        return self.eos_sequence in last_ids

In [ ]:
# 語言模型 class(模型載入、回答輸出、數據資料庫串接)
class Llama3():
    def __init__(self) -> None:
        self.pipe = None
        self.terminators = None
    
    # 載入模型
    def load_model(self):
        # 本地端部屬 llama3 並加入量化
        self.pipe = pipeline("text-generation", model="local",
            model_kwargs={
                "torch_dtype": torch.float16,
                "quantization_config": {"load_in_4bit": True},
                "low_cpu_mem_usage": True,
            },
        )
        self.terminators = [
            self.pipe.tokenizer.eos_token_id,
            self.pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ]
    
    # message json 的打包
    def message(self, info, company_text, stock_data, Q):
        messages = [
            {"role": "system", "content": ""},
            {"role": "user", "content": f"{Q}"},
        ]
        
        return messages
    
    # 回答
    def answer(self, messages:json):
        outputs = self.pipe(
            messages,
            max_new_tokens=1024,  # 減少生成文本的最大長度
            eos_token_id=self.terminators,
            do_sample=True,
            temperature=0.6,
            top_p=0.6,  # 減少top_p值以提高速度
        )
        response = outputs[0]["generated_text"][-1]["content"]
        
        return response
    
    # 抓數據庫資料
    def call_data_deport(self, keywords_pairs):
        def search_stock(stock_id):
            # 查找股票代號對應的行
            df = pd.read_csv(r'data_deport\newest.csv')
            stock_data = df[df["股票代號"] == stock_id]
    
            if stock_data.empty:
                return "未找到該股票代號的資料"
    
            # 提取需要的欄位
            fields = [
                "單月合併營收(千)",
                "單月合併營收年成長(%)", 
                "單月合併營收月變動(%)", 
                "累計合併營收(千)", 
                "近三月合併營收(千)",
                "近三月合併營收年成長(%)",
                "近12月營收合併成長(%)"
            ]

            date = stock_data["年月"].values[0]
            # 取出數據，轉換成字串格式
            result = ', '.join([f"{field}: {stock_data[field].values[0]}" for field in fields])
    
            return result, date

        # 如果沒有股票代碼就直接跑客服
        if len(keywords_pairs) == 0:
            return ''

        # 如果有股票代碼就抓資料庫
        else:
            stock_data = '最新營收資料:\n'
            for i in keywords_pairs:
                # 抓數據找自己要的股號
                try:
                    result_string, date = search_stock(i[0])
                    result_string = f'{i[1]} 最新{date}營收資料:' + result_string
                except:
                    result_string = ''
                # 加入stock_data裡面
                stock_data += result_string
        
        return stock_data

In [ ]:
# 資料預處理 class(第一層預處理(大小寫、停用字)、關鍵字掃描、同義詞轉換)
class DataProcess():
    def __init__(self) -> None:
        self.stocks_df = pd.read_csv('key_syn\stocks.csv')
        self.keywords_content_list = None
        self.keywords_type_list = None
        self.keywords_basic_list = None
        self.keywords_list = None
        self.syn_json = None

    # 大小寫轉換、停用字
    def basic(self, input_text:str):
        # # 去除特殊字元及標點
        # text = re.sub(r'[^\w\s]', '', input_text)

        # 大小寫轉換(全轉成小寫)
        text = re.sub(r'[A-Za-z]+', lambda x: x.group().lower(), input_text)

        return text

    # 抓文本股票代號、名子
    def scan_stock(self, input_text:str):
        # 將 'stock_id' int 轉 str
        self.stocks_df['stock_id'] = self.stocks_df['stock_id'].astype(str)
        # 將 self.stocks_df 轉為字典 用 'stock_id' 作 key 'name' 作 value
        stock_info = dict(zip(self.stocks_df['stock_id'], self.stocks_df['name']))

        # 掃描股號、股名
        keywords_pairs = []
        for code, name in stock_info.items():
            if code in input_text:
                keywords_pairs.append((code, name))
            if name in input_text:
                keywords_pairs.append((code, name))

        # 使用 set 去掉重複項再轉回 list
        keywords_pairs = list(set(map(tuple, keywords_pairs)))
        # 將每對 (code, name) 轉換為列表形式
        keywords_pairs = [list(pair) for pair in keywords_pairs]

        return keywords_pairs
    
    # 載入關鍵字表
    def load_key_syn_list(self):
        # 讀取關鍵字文件
        with open('key_syn\keywords.txt', 'r', encoding='utf-8') as file:
            lines = file.readlines()
            self.keywords_list = [line.strip() for line in lines]

        # 讀取關鍵字文件
        with open('key_syn\keywords_content.txt', 'r', encoding='utf-8') as file:
            lines = file.readlines()
            self.keywords_content_list = [line.strip() for line in lines]

        # 讀取關鍵字文件
        with open('key_syn\keywords_type.txt', 'r', encoding='utf-8') as file:
            lines = file.readlines()
            self.keywords_type_list = [line.strip() for line in lines]

        # 讀取關鍵字文件
        with open('key_syn\keywords_basic.txt', 'r', encoding='utf-8') as file:
            lines = file.readlines()
            self.keywords_basic_list = [line.strip() for line in lines]
        
        # 讀取同義詞 json
        with open('key_syn\synonym_mapping.json', 'r', encoding='utf-8') as json_file:
            self.syn_json = json.load(json_file)

    # 掃描關鍵字
    def scan_type_keyword(self, input_text:str):
        extracted_keywords = []
        for keyword in self.keywords_type_list:
            if keyword in input_text:
                extracted_keywords.append(keyword)

        return extracted_keywords
    
    # 掃描關鍵字
    def scan_content_keyword(self, input_text:str):
        extracted_keywords = []
        for keyword in self.keywords_content_list:
            if keyword in input_text:
                extracted_keywords.append(keyword)

        return extracted_keywords
    
    def scan_basic_keyword(self, input_text:str):
        extracted_keywords = []
        for keyword in self.keywords_basic_list:
            if keyword in input_text:
                extracted_keywords.append(keyword)

        return extracted_keywords
    
    # 同義詞轉換
    def convert_synonym(self, extracted_keywords: list):
        unique_keywords = set()
        for keyword in extracted_keywords:
            transformed_keyword = self.syn_json.get(keyword, keyword)
            unique_keywords.add(transformed_keyword)
        return list(unique_keywords)

In [ ]:
basic_text = '' 

In [ ]:
# Line運作@app (抓ID)
# 帳務
access_token = ''
secret = ''
# 確認 token secret
line_bot_api = LineBotApi(access_token)
handler = WebhookHandler(secret)

# 眾多的 class
CD = ClientDeport()
DP = DataProcess()
DP.load_key_syn_list()
ES = ELS()
ES.login()
ES.load_key_syn_list()
LL = Llama3()
LL.load_model()

def schedule_task(interval):
    """
    每隔指定時間 interval(秒) 來執行 func 函式
    """
    while True:
        time.sleep(interval)
        CD.get_client_df()
        CD.get_iso_data_list()

app = Flask(__name__)

@app.route("/", methods=['POST'])
def linebot():
    # 取得訊息內容
    body = request.get_data(as_text=True)
    try:
        # 訊息轉 json
        json_data = json.loads(body)
        # header 處理
        signature = request.headers['X-Line-Signature']
        handler.handle(body, signature)
        
        # 取得回傳訊息的 Token
        tk = json_data['events'][0]['replyToken']
        # 取得 LINE 收到的訊息類型
        type = json_data['events'][0]['message']['type']
        # 取得 user_id
        user_id = json_data['events'][0]['source']['userId']

        if type=='text':
            # 取得 LINE 收到的文字訊息
            msg = json_data['events'][0]['message']['text']

            # 刪除空格及換行
            msg_cleaned = re.sub(r'\s+', '', msg)
            # 檢查開頭是否有 ""
            prefix = ""
            if msg_cleaned.startswith(prefix):
                msg_cleaned = msg_cleaned[3:]
                # ---------------------------------
                # 1.比對ID
                # 比對文字
                name, phone = CD.extract_name_phone(msg_cleaned)
                
                # 認證異常請她連絡理專
                if CD.ID_check(user_id) == '認證異常':
                    line_bot_api.reply_message(tk, TextSendMessage('認證異常'))
                    
                # 無資料比對msg比對成功建檔並接續語言模型 失敗就請他回傳正確格式
                elif CD.ID_check(user_id) == '尚無資料' and name == None:
                    line_bot_api.reply_message(tk, TextSendMessage('請先依格式給予 姓名_手機號碼'))
                    
                # 其他的為認證成功或初次建檔成功
                else:
                    # 初建資料庫的
                    if user_id not in CD.ID_list:
                        print(f'新建資料{user_id}')
                        CD.renew_client_df(name, phone, user_id)
                        CD.bulid_iso_data(user_id)
                        CD.get_client_df()
                        CD.get_iso_data_list()
                        line_bot_api.reply_message(tk, TextSendMessage(f'初建檔案姓名:{name} 電話:{phone}'))
                    
                    else:
                        # 2.文字處理
                        input_text = DP.basic(msg_cleaned)
                        scan_stock_list = DP.scan_stock(msg_cleaned)
                        scan_type_keywords_list = DP.scan_type_keyword(msg_cleaned)
                        scan_type_keywords_list = DP.convert_synonym(scan_type_keywords_list)
                        scan_content_keywords_list = DP.scan_content_keyword(msg_cleaned)
                        scan_content_keywords_list = DP.convert_synonym(scan_content_keywords_list)
                        scan_keywords_list = scan_content_keywords_list + scan_type_keywords_list
                        scan_basic_keywords_list = DP.scan_basic_keyword(msg_cleaned)
                        scan_basic_keywords_list = DP.convert_synonym(scan_basic_keywords_list)
                        
                        # 3.更新客戶資料庫
                        CD.renew_iso_data(user_id, input_text, scan_keywords_list)

                        # 4.文本、數據資料庫
                        best_doc, score, time = ES.search('txtdeport', scan_keywords_list, scan_content_keywords_list, scan_type_keywords_list, msg_cleaned)
                        print(best_doc)
                        if scan_basic_keywords_list != []:
                            info = basic_text
                        else:
                            info = ''

                        stock_data_txt = LL.call_data_deport(scan_stock_list)
                        print(stock_data_txt)
                        # 5.語言模型
                        message = LL.message(info, best_doc, stock_data_txt, input_text)
                        answer = LL.answer(message)

                        # 回傳訊息
                        print(answer)
                        line_bot_api.reply_message(tk, TextSendMessage(answer))
            else:
                pass
        else:
            pass
    # 發生錯誤 print 問題
    except:
        print('error', body)

    # 驗證 Webhook 使用，不能省略
    return 'OK'

if __name__ == "__main__":
    # 開啟一個新的執行緒來每1分鐘執行一次 load_key_syn_list
    thread = threading.Thread(target=schedule_task, args=(300,))
    thread.daemon = True
    thread.start()
    
    # 啟動 Flask 應用
    # app.run()
    serve(app, host='0.0.0.0', port=5000)